# Evaluate SatRain retrievals

This notebook evaluates SatRain ML retrievals across all testing datasets.

In [3]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from typing import Tuple

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ERA5 Data

In [13]:
from satrain.input import Ancillary
anc = Ancillary(variables=["total_precipitation"])

def retrieve_era5(input_data: xr.Dataset) -> xr.Dataset:
    """
    Retrieval callback function to load GPROF data corresponding to IPWGML SPR evaluation data.

    Args:
        input_data: An xarray.Dataset containing the retrieval input data.

    Return:
        An xarray.Dataset containing the retrieval results.
    """
    return xr.Dataset({
        "surface_precip": 1e3 * input_data[{"features_ancillary": 0}].ancillary
    })

## Evaluation

In [14]:
from satrain.evaluation import Evaluator
from satrain.target import TargetConfig

for domain in ["austria", "conus", "korea"]:
    evaluator = Evaluator(
        domain=domain,
        base_sensor="gmi",
        geometry="gridded",
        retrieval_input=[{"name": "ancillary", "variables": ["total_precipitation"]}]
    )
    evaluator.evaluate(retrieve_era5, n_processes=8)
    results = evaluator.get_results()
    results.to_netcdf(f'results_era5_{domain}.nc')

Output()

Output()

Output()